In [1]:
import sys
sys.path.append(r"C:\Users\preda\vs_files\Lucknow-LLM\lucknowllm")
print(sys.path)
#from lucknowllm.models.gemini_model import GeminiModel
from langchain_community.embeddings import HuggingFaceEmbeddings
# from models.gemini_model import GeminiModel
from langchain import hub
from langchain_community.vectorstores import Weaviate
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnablePassthrough
from langchain_pinecone import PineconeVectorStore
import os
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.prompts import PromptTemplate
from langchain.chains import create_history_aware_retriever, create_retrieval_chain,StuffDocumentsChain, LLMChain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.memory import VectorStoreRetrieverMemory,ConversationSummaryBufferMemory,CombinedMemory
from langchain.chains import ConversationChain,ConversationalRetrievalChain
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

['c:\\Users\\preda\\vs_files\\Lucknow-LLM\\Lallan', 'c:\\Users\\preda\\.conda\\envs\\lallan\\python311.zip', 'c:\\Users\\preda\\.conda\\envs\\lallan\\DLLs', 'c:\\Users\\preda\\.conda\\envs\\lallan\\Lib', 'c:\\Users\\preda\\.conda\\envs\\lallan', '', 'C:\\Users\\preda\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\preda\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\preda\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\preda\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\preda\\.conda\\envs\\lallan\\Lib\\site-packages', 'C:\\Users\\preda\\vs_files\\Lucknow-LLM\\lucknowllm']


c:\Users\preda\.conda\envs\lallan\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import google.generativeai as genai

print('Available base models:', [m.name for m in genai.list_models()])
print('My tuned models:', [m.name for m in genai.list_tuned_models()])

Available base models: ['models/chat-bison-001', 'models/text-bison-001', 'models/embedding-gecko-001', 'models/gemini-1.0-pro', 'models/gemini-1.0-pro-001', 'models/gemini-1.0-pro-latest', 'models/gemini-1.0-pro-vision-latest', 'models/gemini-1.5-pro-latest', 'models/gemini-pro', 'models/gemini-pro-vision', 'models/embedding-001', 'models/text-embedding-004', 'models/aqa']


Unauthenticated: 401 API keys are not supported by this API. Expected OAuth2 access token or other authentication credentials that assert a principal. See https://cloud.google.com/docs/authentication [reason: "CREDENTIALS_MISSING"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
metadata {
  key: "method"
  value: "google.ai.generativelanguage.v1beta.ModelService.ListTunedModels"
}
]

In [3]:
from langchain_pinecone import PineconeVectorStore
embeddings = HuggingFaceEmbeddings()
index_name = "lai-rag"



In [25]:
safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_ONLY_HIGH"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_ONLY_HIGH"},
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_ONLY_HIGH",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_ONLY_HIGH",
    },
]

llm=ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.5,convert_system_message_to_human=True,safety_settings=safety_settings)

ValidationError: 1 validation error for ChatGoogleGenerativeAI
safety_settings -> __key__
  value is not a valid integer (type=type_error.integer)

In [5]:
docsearch = PineconeVectorStore( index_name=index_name,embedding=embeddings)

In [6]:
_DEFAULT_TEMPLATE = """You are an expert informator system about Lucknow,you will be given questions and context and you'll return the FINAL answer in a sweet and sarcastic tone containing the content of the Observation you made . You will use Hum instead of main. Your name is Lallan. The full form of Lallan is 'Lucknow Artificial Language and Learning Assistance Network'. Call only Janab-e-Alaa instead of phrase My dear Friend. Say Salaam= Miya! instead of Greetings and dont greet in every answer if once done .You perform question-answering tasks.Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know..The following is a friendly conversation between a human and an Lallan AI.lallan is talkative and provides lots of specific details from its context. If the lallan does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
{history}

(use this information to form the response to the current question)
previous conversation:
{conversation_summary}
Current conversation:
Human: {input}
AI:"""

In [6]:

### Answer question ###
qa_system_prompt = """You are an expert informator system about Lucknow,you will be given questions and context and you'll return the FINAL answer in a sweet and sarcastic tone containing the content of the Observation you made . You will use Hum instead of main. Your name is Lallan. The full form of Lallan is 'Lucknow Artificial Language and Learning Assistance Network'. Call only Janab-e-Alaa instead of phrase My dear Friend. Say Salaam= Miya! instead of Greetings and dont greet in every answer if once done .You perform question-answering tasks.Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. give full explanatory answer if needed.

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [7]:
qa_prompt.get_prompts

<bound method Runnable.get_prompts of ChatPromptTemplate(input_variables=['chat_history', 'context', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="You are an expert informator system about Lucknow,you will be given questions and context and you'll return the FINAL answer in a sweet and sarcastic tone containing the content of the Observation you made . You will use Hum instead of main. Your name is Lallan. The full form of Lallan is 'Lucknow Artificial Language and Learning Assistance Network'. Call only Janab-e-Alaa instead of phrase My dear Friend. Say Salaam= Miya! instead of Greetings and dont greet in ever

In [8]:
### Contextualize question ###
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, docsearch.as_retriever(
            search_type="similarity_score_threshold",
            search_kwargs={"k": 6, "score_threshold": 0.7}), contextualize_q_prompt
)


In [9]:
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [10]:
store = {}

In [11]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


In [12]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [23]:
conversational_rag_chain.invoke(
    {"input": "mera naam kya hai?"},
    config={
        "configurable": {"session_id": "abc123"}
    }  # constructs a key "abc123" in `store`.
)["answer"]

c:\Users\preda\.conda\envs\lallan\Lib\site-packages\langchain_google_genai\chat_models.py:308: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 Developer instruction is not enabled for models/gemini-pro

In [14]:
store["abc123"]

ChatMessageHistory(messages=[HumanMessage(content='mera naam kya hai?'), AIMessage(content='मुझे आपका नाम नहीं पता क्योंकि आपने मुझे वह जानकारी नहीं दी है।')])